In [29]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import reset_ramsey

# Load the dataset

In [15]:
url = "https://faculty.utrgv.edu/diego.escobari/teaching/Datasets/401KSUBS.xls"
data = pd.read_excel(url, header=None)


# Fix column names manually

In [16]:
columns = ['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'p401k', 'pira', 'incsq', 'agesq']
data.columns = columns

# Display first 10 rows in a clean table format

In [17]:
print("First 10 rows of the dataset:")
print(data.head(10).to_string(index=False))

First 10 rows of the dataset:
 e401k    inc  marr  male  age  fsize  nettfa  p401k  pira     incsq  agesq
     0 13.170     0     0   40      1   4.575      0     1  173.4489   1600
     1 61.230     0     1   35      1 154.000      1     0 3749.1130   1225
     0 12.858     1     0   44      2   0.000      0     0  165.3282   1936
     0 98.880     1     1   44      2  21.800      0     0 9777.2540   1936
     0 22.614     0     0   53      1  18.450      0     0  511.3930   2809
     0 15.000     1     0   60      3   0.000      0     0  225.0000   3600
     0 37.155     1     0   49      5   3.483      0     1 1380.4940   2401
     0 31.896     1     0   38      5  -2.100      0     0 1017.3550   1444
     0 47.295     1     0   52      2   5.290      0     1 2236.8170   2704
     1 29.100     0     1   45      1  29.600      0     1  846.8100   2025


#  Fit Model With Highly Correlated Variable

In [18]:
X1 = data[['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'pira', 'incsq', 'agesq']]
y = data['p401k']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y, X1).fit()


In [24]:
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  p401k   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     1394.
Date:                Wed, 08 Oct 2025   Prob (F-statistic):               0.00
Time:                        09:59:38   Log-Likelihood:                -1436.9
No. Observations:                9275   AIC:                             2896.
Df Residuals:                    9264   BIC:                             2974.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0602      0.050      1.203      0.2

 # Checking Correlation

Choose one independent variable (educ)

In [19]:
independent_var = 'inc'
correlation = data.corr()[independent_var]
print(f"\nCorrelation of '{independent_var}' with other variables:")
print(correlation)


Correlation of 'inc' with other variables:
e401k     0.268178
inc       1.000000
marr      0.362008
male     -0.069871
age       0.105638
fsize     0.110170
nettfa    0.376586
p401k     0.270833
pira      0.364354
incsq     0.940161
agesq     0.087305
Name: inc, dtype: float64


#  Fit Model Without Highly Correlated Variable

In [20]:
X2 = data[['e401k', 'inc', 'marr', 'male', 'age', 'fsize', 'nettfa', 'pira', 'agesq']]
X2 = sm.add_constant(X2)
model2 = sm.OLS(y, X2).fit()


# Fit OLS model

In [23]:
print("\nOLS Regression Summary (after omitting highly correlated variable):")
print(model2.summary())


OLS Regression Summary (after omitting highly correlated variable):
                            OLS Regression Results                            
Dep. Variable:                  p401k   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     1549.
Date:                Wed, 08 Oct 2025   Prob (F-statistic):               0.00
Time:                        09:59:09   Log-Likelihood:                -1437.0
No. Observations:                9275   AIC:                             2894.
Df Residuals:                    9265   BIC:                             2965.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------


#  R² Score

In [26]:
print("\nR² Score:", model2.rsquared)


R² Score: 0.6007650561569755


# RESET Test for Model Specification

In [30]:
reset_test = reset_ramsey(model2, degree=2)
print("\nRESET Test Result:")
print("F-statistic:", reset_test.fvalue)
print("p-value:", reset_test.pvalue)


RESET Test Result:
F-statistic: 132.51368773698343
p-value: 1.860126673386771e-30


In [31]:
if reset_test.pvalue > 0.05:
    print("Model specification is okay (fail to reject null).")
else:
    print("Model may be mis-specified (reject null).")

Model may be mis-specified (reject null).


# Compare R² values in a clean table

In [32]:

comparison = pd.DataFrame({
    'Model Description': ['Model 1 (All Variables)', 'Model 2 (Without incsq)'],
    'Variables Included': ['With incsq', 'Without incsq'],
    'R² Value': [model1.rsquared, model2.rsquared]
})

print("Model Comparison Table:")
print(comparison.to_string(index=False))

Model Comparison Table:
      Model Description Variables Included  R² Value
Model 1 (All Variables)         With incsq  0.600772
Model 2 (Without incsq)      Without incsq  0.600765
